In [43]:
import pandas as pd
from tensorflow import keras as ks
import numpy as np
from tensorflow.keras import Input
import tensorflow as tf

In [161]:
train = pd.read_csv('../data/split_data/train_stats.csv', index_col=0)
test = pd.read_csv('../data/split_data/test_stats.csv', index_col=0)
train.head()

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y40,y41,y42,y43,y44,y45,y46,y47,y48,y49
0,1.464797e+08,0.227959,0.032626,0.243542,13.396406,0.0,0.038905,0.667826,20.925667,0.614147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.464797e+08,0.227959,0.032801,0.243854,13.451273,0.0,0.039181,0.667826,20.925667,0.614147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.464797e+08,0.227959,0.033088,0.244401,13.538579,0.0,0.039247,0.667826,20.925667,0.614147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.464797e+08,0.227959,0.033308,0.244818,13.605041,0.0,0.039266,0.667826,20.925667,0.614147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.464797e+08,0.227959,0.033349,0.244896,13.617806,0.0,0.039219,0.667826,20.925667,0.614147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
# обучаем модель только на не аномальных данных
zero_train = train.copy()

for column in zero_train.iloc[:, -50:].columns:
    zero_train = zero_train.loc[zero_train[column] == 0]

zero_train.iloc[0:, -100: -50]

,r-r-0,r-r-1,r-r-2,r-r-3,r-r-4,r-r-5,r-r-6,r-r-7,r-r-8,r-r-9,...,r-r-40,r-r-41,r-r-42,r-r-43,r-r-44,r-r-45,r-r-46,r-r-47,r-r-48,r-r-49
0,0.253906,0.247396,0.251302,0.260417,0.255208,0.263021,0.291667,0.305990,0.304687,0.295573,...,0.239583,0.238281,0.227865,0.226562,0.227865,0.230469,0.227865,0.236979,0.242188,0.251302
1,0.247396,0.251302,0.260417,0.255208,0.263021,0.291667,0.305990,0.304687,0.295573,0.265625,...,0.238281,0.227865,0.226562,0.227865,0.230469,0.227865,0.236979,0.242188,0.251302,0.269531
2,0.251302,0.260417,0.255208,0.263021,0.291667,0.305990,0.304687,0.295573,0.265625,0.251302,...,0.227865,0.226562,0.227865,0.230469,0.227865,0.236979,0.242188,0.251302,0.269531,0.274740
3,0.260417,0.255208,0.263021,0.291667,0.305990,0.304687,0.295573,0.265625,0.251302,0.246094,...,0.226562,0.227865,0.230469,0.227865,0.236979,0.242188,0.251302,0.269531,0.274740,0.272135
4,0.255208,0.263021,0.291667,0.305990,0.304687,0.295573,0.265625,0.251302,0.246094,0.233073,...,0.227865,0.230469,0.227865,0.236979,0.242188,0.251302,0.269531,0.274740,0.272135,0.264323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,0.195312,0.197917,0.197917,0.199219,0.196615,0.196615,0.195312,0.197917,0.196615,0.194010,...,0.196615,0.195312,0.395833,0.195312,0.195312,0.391927,0.191406,0.194010,0.192708,0.192708
3588,0.197917,0.197917,0.199219,0.196615,0.196615,0.195312,0.197917,0.196615,0.194010,0.192708,...,0.195312,0.395833,0.195312,0.195312,0.391927,0.191406,0.194010,0.192708,0.192708,0.191406
3589,0.197917,0.199219,0.196615,0.196615,0.195312,0.197917,0.196615,0.194010,0.192708,0.192708,...,0.395833,0.195312,0.195312,0.391927,0.191406,0.194010,0.192708,0.192708,0.191406,0.191406
3590,0.199219,0.196615,0.196615,0.195312,0.197917,0.196615,0.194010,0.192708,0.192708,0.188802,...,0.195312,0.195312,0.391927,0.191406,0.194010,0.192708,0.192708,0.191406,0.191406,0.191406


In [163]:
r_r_series_input = Input(shape=(50, 1))
r_r_series_layer = ks.layers.Conv1D(filters=32,
                                    kernel_size=10,
                                    padding='same',
                                    #strides=2,
                                    activation='relu')(r_r_series_input)
r_r_series_layer = ks.layers.Dropout(0.2)(r_r_series_layer)
r_r_series_layer = ks.layers.Conv1D(filters=16,
                                    kernel_size=10,
                                    padding='same',
                                    #strides=2,
                                    activation='relu')(r_r_series_layer)
r_r_series_layer = ks.layers.Dropout(0.2)(r_r_series_layer)

r_r_series_decoder_layer = ks.layers.Conv1DTranspose(filters=16,
                                                     kernel_size=10,
                                                     padding="same",
                                                     #strides=2,
                                                     activation="relu")(r_r_series_layer)
r_r_series_decoder_layer = ks.layers.Dropout(0.2)(r_r_series_decoder_layer)

r_r_series_decoder_layer = ks.layers.Conv1DTranspose(filters=32,
                                                     kernel_size=10,
                                                     padding="same",
                                                     #strides=2,
                                                     activation="relu")(r_r_series_decoder_layer)
r_r_series_decoder_layer = ks.layers.Conv1DTranspose(filters=1,
                                                     kernel_size=10,
                                                     padding="same")(r_r_series_decoder_layer)


r_r_series_model = ks.Model(inputs=r_r_series_input,

                            outputs=r_r_series_decoder_layer)

In [164]:
r_r_series_model.compile(optimizer='adam',
                         loss='mse',
                         metrics='mae')

In [165]:
r_r_series_model.fit(batch_size=16,
                     x=zero_train.iloc[:, 24:-50].values.reshape(-1,50,1),
                     y=zero_train.iloc[:, 24:-50].values.reshape(-1,50,1),
                     validation_data=(test.iloc[:, 24:-50].values.reshape(-1,50,1),
                                      test.iloc[:, 24:-50].values.reshape(-1,50,1)),
                     epochs=20)


Epoch 1/20
136/136 [==============================] - 4s 18ms/step - loss: 0.0120 - mae: 0.0773 - val_loss: 0.0011 - val_mae: 0.0174

In [142]:
# запихиваем реальные данные и предсказанные -> класифицируем

anomaly_detection_r_r_input = Input(shape=(50, 2))
anomaly_detection_r_r_layer = ks.layers.Dense(8, activation='tanh')(anomaly_detection_r_r_input)
anomaly_detection_r_r_layer = ks.layers.Dense(16, activation='tanh')(anomaly_detection_r_r_layer)
anomaly_detection_r_r_layer = ks.layers.Dense(8, activation='tanh')(anomaly_detection_r_r_layer)
anomaly_detection_r_r_layer = ks.layers.Dense(2, activation='tanh')(anomaly_detection_r_r_layer)
anomaly_detection_r_r_model= ks.Model(inputs=anomaly_detection_r_r_input,
                                  outputs=anomaly_detection_r_r_layer)

anomaly_detection_stat_input = Input(shape=12)
anomaly_detection_stat_layer = ks.layers.Dense(32, activation='tanh')(anomaly_detection_stat_input)
anomaly_detection_stat_layer = ks.layers.Dense(64, activation='tanh')(anomaly_detection_stat_layer)
anomaly_detection_stat_model = ks.Model(inputs=anomaly_detection_stat_input,
                                        outputs=anomaly_detection_stat_layer)

conc_layer = ks.layers.concatenate(inputs=[anomaly_detection_r_r_model.output,
                                           anomaly_detection_stat_model.output])


conc_layer = ks.layers.Dense(32, activation='tanh')(conc_layer)
conc_layer = ks.layers.Dense(16, activation='tanh')(conc_layer)
conc_layer = ks.layers.Dense(8, activation='tanh')(conc_layer)
conc_layer = ks.layers.Dense(2, activation='tanh')(conc_layer)

anomaly_detection_model = ks.Model(inputs=[anomaly_detection_r_r_model.input,
                                           anomaly_detection_stat_model.input],
                                   outputs=conc_layer)


In [143]:
anomaly_detection_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [167]:
test_r_r_series = test.iloc[:, 24:-50].values.reshape(-1,50,1)
predict_r_r_series = r_r_series_model.predict(test_r_r_series)

predict_r_r_series.shape




(439, 50, 1)

In [145]:
anomaly_series = test.iloc[:, -50:].values.reshape(-1)
anomaly_series = ks.utils.to_categorical(anomaly_series, 2)

In [153]:
test.iloc[:, :12]


,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,idm_int,cat_int
170,7.128949e+08,0.181276,0.031003,0.185625,16.702170,0.0,0.045401,2.584615,20.368426,3.861514,73.220006,0.956023
171,6.926739e+08,0.181237,0.030996,0.185729,16.688956,0.0,0.045406,2.510769,20.372761,3.751983,73.244256,0.928400
172,6.724479e+08,0.181198,0.030989,0.185807,16.678028,0.0,0.045404,2.436923,20.377208,3.642426,73.257746,0.900929
173,6.522167e+08,0.181157,0.030982,0.185938,16.662595,0.0,0.045520,2.363077,20.381773,3.532841,73.462787,0.871189
174,6.319803e+08,0.181116,0.030974,0.186016,16.651400,0.0,0.045230,2.289231,20.386461,3.423227,73.011703,0.849179
...,...,...,...,...,...,...,...,...,...,...,...,...
423,6.027208e+06,31.222656,0.038203,0.191823,19.915814,0.0,0.047164,3.763709,0.162898,0.023701,61.727761,1.198812
424,9.673375e+08,0.184751,0.038292,0.191615,19.983698,0.0,0.055697,3.574340,26.144257,4.005382,72.727463,1.017497
425,9.671513e+08,0.184787,0.038329,0.192109,19.951433,0.0,0.055857,3.574340,26.139223,4.004611,72.866642,1.015554
426,9.414538e+08,0.185172,0.046836,0.196094,23.884424,0.0,0.069547,3.486626,25.444698,3.996275,74.245790,0.996690


In [146]:
data_classification = pd.DataFrame(columns=['true_r_r', 'pred_r_r'])
data_classification['true_r_r'] = test_r_r_series
data_classification['pred_r_r'] = predict_r_r_series

In [147]:
anomaly_detection_model.fit(x=[data_classification.values,
                               test.iloc[:, :12].values.reshape(-1)
                               ],
                            y=anomaly_series,
                            epochs=15)

ValueError: Data cardinality is ambiguous:
  x sizes: 10400, 2496
  y sizes: 10400
Make sure all arrays contain the same number of samples.

In [137]:
pred_list = []
pred = anomaly_detection_model.predict(data_classification.values)
for row in pred:
    if row[1] > 0.33:
        pred_list.append(1)
    else:
        pred_list.append(0)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [138]:
from sklearn.metrics import confusion_matrix, f1_score

In [139]:
f1_score(y_true=np.argmax(anomaly_series, axis=1), y_pred=pred_list, average='micro')

0.7644230769230768

In [140]:
confusion_matrix(y_true=np.argmax(anomaly_series, axis=1), y_pred=pred_list)

array([[7950,    0],
       [2450,    0]], dtype=int64)